In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format

In [2]:
import pandas as pd
import math
import seaborn as sns
from matplotlib import pyplot as plt
import re
import numpy as np
import geopandas as gpd
from shapely.geometry import Point # Point class
from shapely.geometry import shape
import io
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")
from geopy.geocoders import Nominatim
from time import sleep
from datetime import datetime

C:\Users\Md Mintu Miah\anaconda3\envs\geo_env\lib\site-packages\geopandas\_compat.py:112: UserWarning: The Shapely GEOS version (3.11.0-CAPI-1.17.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [39]:
madt=pd.read_csv('C:/UC Berkeley Project Backup/AT Census/Processed Data/permanent_counter_2018_2021_MADT_data.csv')
madt.head()

,Unnamed: 0,ID,location,latitude,longitude,year,month,MTT,days_in_month,Simple_MADT,AASHTO_MADT,diff,type
0,0,3575,urban,32.66,-117.07,2018,3,4984.00,31,160.77,237.33,-76.56,eco_permanent
1,1,3575,urban,32.66,-117.07,2018,4,8473.00,30,282.43,302.61,-20.17,eco_permanent
2,2,3575,urban,32.66,-117.07,2018,5,5853.00,31,188.81,278.71,-89.91,eco_permanent
3,3,3575,urban,32.66,-117.07,2018,6,6068.00,30,202.27,288.95,-86.69,eco_permanent
4,4,3575,urban,32.66,-117.07,2018,7,7991.00,31,257.77,285.39,-27.62,eco_permanent


In [40]:
loc=madt[['ID','year','latitude','longitude','location']]
loc1=loc.drop_duplicates(subset=['ID','latitude','longitude'], keep='first')
loc1.shape

(282, 5)

In [41]:
# bring the location based on lat and long
def city_state_country(row):
    coord = f"{row['latitude']}, {row['longitude']}"
    sleep(1)
    location = geolocator.reverse(coord, exactly_one=True, language='en')
    if not location:
        # if you see many in a row, it's probably Nominatim blocking you.
        # if it's just every once in a while, there were just some bad results. 
        print('Failed with coord: ', coord)
        row['road'],row['city'],row['county'], row['state'], row['country'] = None, None, None
        return row
    address = location.raw['address']
    city = address.get('city', '')
    county=address.get('county', '')
    state = address.get('state', '')
    country = address.get('country', '')
    row['road']=address.get('road', '')
    row['city'] = city
    row['county'] = county
    row['state'] = state
    row['country'] = country
    return row

In [42]:
loc2 = loc1.apply(city_state_country, axis=1)
loc2.head()

,ID,year,latitude,longitude,location,road,city,county,state,country
0,3575,2018,32.66,-117.07,urban,Sweetwater Bikeway,Chula Vista,San Diego County,California,United States
25,59064,2018,32.77,-117.02,urban,University Avenue,,CAL Fire San Diego Unit,California,United States
66,43453,2018,32.77,-117.02,urban,University Avenue,,CAL Fire San Diego Unit,California,United States
112,3578,2018,32.58,-117.13,urban,Palm Avenue,,San Diego County,California,United States
133,19208,2018,32.59,-117.11,urban,Bayshore Bikeway,,San Diego County,California,United States


In [43]:
loc3=loc2.drop(['year','location'],axis=1)

In [62]:
madt1=pd.merge(madt, loc3, on=['ID',"latitude",'longitude'], how='inner')
madt1.head()

,Unnamed: 0,ID,location,latitude,longitude,year,month,MTT,days_in_month,Simple_MADT,AASHTO_MADT,diff,type,road,city,county,state,country
0,0,3575,urban,32.66,-117.07,2018,3,4984.00,31,160.77,237.33,-76.56,eco_permanent,Sweetwater Bikeway,Chula Vista,San Diego County,California,United States
1,1,3575,urban,32.66,-117.07,2018,4,8473.00,30,282.43,302.61,-20.17,eco_permanent,Sweetwater Bikeway,Chula Vista,San Diego County,California,United States
2,2,3575,urban,32.66,-117.07,2018,5,5853.00,31,188.81,278.71,-89.91,eco_permanent,Sweetwater Bikeway,Chula Vista,San Diego County,California,United States
3,3,3575,urban,32.66,-117.07,2018,6,6068.00,30,202.27,288.95,-86.69,eco_permanent,Sweetwater Bikeway,Chula Vista,San Diego County,California,United States
4,4,3575,urban,32.66,-117.07,2018,7,7991.00,31,257.77,285.39,-27.62,eco_permanent,Sweetwater Bikeway,Chula Vista,San Diego County,California,United States


In [99]:
check=madt1.drop_duplicates(subset=['ID'], keep='first')
check.shape

(282, 18)

In [45]:
# save the data
madt1.to_csv('C:/UC Berkeley Project Backup/AT Census/Processed Data/permanent_counter_2018_2021_MADT_with_location_data.csv')

# Now Bring theBicycle Facilities, Strava and Merge with Eco Counters

In [119]:
# Import Bike Facilities
madt_fac=pd.read_csv('C:/UC Berkeley Project Backup/AT Census/Processed Data/Eco_MADT_Facilities1.csv')
madt_fac.head()

,ID,location,latitude,longitude,year,month,MTT,days_in_month,Simple_MADT,AASHTO_MADT,...,long,name,OSM Functional Class,City Bicycle Facilities,geometry,STATEFP,COUNTYFP,COUNTYNS,GEOID,Location_County
0,3575,urban,32.66,-117.07,2018,3,4984,31,160.77,237.33,...,-117.07,Sweetwater Bikeway,cycleway,Path,POINT (-117.070625 32.657333),6,73,277301,6073,San Diego
1,3575,urban,32.66,-117.07,2018,4,8473,30,282.43,302.61,...,-117.07,Sweetwater Bikeway,cycleway,Path,POINT (-117.070625 32.657333),6,73,277301,6073,San Diego
2,3575,urban,32.66,-117.07,2018,5,5853,31,188.81,278.71,...,-117.07,Sweetwater Bikeway,cycleway,Path,POINT (-117.070625 32.657333),6,73,277301,6073,San Diego
3,3575,urban,32.66,-117.07,2018,6,6068,30,202.27,288.95,...,-117.07,Sweetwater Bikeway,cycleway,Path,POINT (-117.070625 32.657333),6,73,277301,6073,San Diego
4,3575,urban,32.66,-117.07,2018,7,7991,31,257.77,285.39,...,-117.07,Sweetwater Bikeway,cycleway,Path,POINT (-117.070625 32.657333),6,73,277301,6073,San Diego


In [157]:
madt_fac.shape

(6295, 32)

In [164]:
madt_fac.dtypes

ID                           int64
location                    object
latitude                   float64
longitude                  float64
year                         int64
month                        int64
MTT                          int64
days_in_month                int64
Simple_MADT                float64
AASHTO_MADT                float64
diff                       float64
type                        object
road                        object
city                        object
county                      object
state                       object
country                     object
id                           int64
serial_num                  object
user_type                   object
site_name                   object
lat                        float64
long                       float64
name                        object
OSM Functional Class        object
City Bicycle Facilities     object
geometry                    object
STATEFP                      int64
COUNTYFP            

In [155]:
# Read Strava Data
stv18=pd.read_csv('C:/UC Berkeley Project Backup/AT Census/Processed Data/eco_strava_monthly_matching2018.csv')
stv18v=stv18[['id', 'lat', 'long', 'Region', 'edgeUID_', 'osmId',
       'dist', 'edge_uid', 'osm_refere',  'month',
       'stv_total', 'stv_com_total',
       'stv_rec_total', 'stv_com_ratio', 'stv_rec_ratio']]
stv19=pd.read_csv('C:/UC Berkeley Project Backup/AT Census/Processed Data/eco_strava_monthly_matching2019.csv')
stv19v=stv19[['id', 'lat', 'long', 'Region', 'edgeUID_', 'osmId',
       'dist', 'edge_uid', 'osm_refere',  'month',
       'stv_total', 'stv_com_total',
       'stv_rec_total', 'stv_com_ratio', 'stv_rec_ratio']]
stv20=pd.read_csv('C:/UC Berkeley Project Backup/AT Census/Processed Data/eco_strava_monthly_matching2019.csv')
stv20v=stv20[['id', 'lat', 'long', 'Region', 'edgeUID_', 'osmId',
       'dist', 'edge_uid', 'osm_refere',  'month',
       'stv_total', 'stv_com_total',
       'stv_rec_total', 'stv_com_ratio', 'stv_rec_ratio']]
stv21=pd.read_csv('C:/UC Berkeley Project Backup/AT Census/Processed Data/eco_strava_monthly_matching2019.csv')
stv21v=stv20[['id', 'lat', 'long', 'Region', 'edgeUID_', 'osmId',
       'dist', 'edge_uid', 'osm_refere',  'month',
       'stv_total', 'stv_com_total',
       'stv_rec_total', 'stv_com_ratio', 'stv_rec_ratio']]
Stv=pd.concat([stv18v,stv19v,stv20v,stv21v],axis=0)
Stv.head()

,id,lat,long,Region,edgeUID_,osmId,dist,edge_uid,osm_refere,month,stv_total,stv_com_total,stv_rec_total,stv_com_ratio,stv_rec_ratio
0,100000671,37.79,-122.46,Bay Area,308723101,215845474,d4,308723101,215845474,2018-08,420,20,400,0.05,0.95
1,100000671,37.79,-122.46,Bay Area,308723101,215845474,d4,308723101,215845474,2018-12,385,10,375,0.03,0.97
2,100000671,37.79,-122.46,Bay Area,308723101,215845474,d4,308723101,215845474,2018-01,395,25,370,0.06,0.94
3,100000671,37.79,-122.46,Bay Area,308723101,215845474,d4,308723101,215845474,2018-11,270,15,255,0.06,0.94
4,100000671,37.79,-122.46,Bay Area,308723101,215845474,d4,308723101,215845474,2018-04,430,30,400,0.07,0.93


In [160]:
Stv[[ "year","month"]] = Stv["month"].str.split("-", expand = True)
Stv.head()

,id,lat,long,Region,edgeUID_,osmId,dist,edge_uid,osm_refere,month,stv_total,stv_com_total,stv_rec_total,stv_com_ratio,stv_rec_ratio,year
0,100000671,37.79,-122.46,Bay Area,308723101,215845474,d4,308723101,215845474,08,420,20,400,0.05,0.95,2018
1,100000671,37.79,-122.46,Bay Area,308723101,215845474,d4,308723101,215845474,12,385,10,375,0.03,0.97,2018
2,100000671,37.79,-122.46,Bay Area,308723101,215845474,d4,308723101,215845474,01,395,25,370,0.06,0.94,2018
3,100000671,37.79,-122.46,Bay Area,308723101,215845474,d4,308723101,215845474,11,270,15,255,0.06,0.94,2018
4,100000671,37.79,-122.46,Bay Area,308723101,215845474,d4,308723101,215845474,04,430,30,400,0.07,0.93,2018


In [170]:
Stv["year"]=Stv["year"].astype(np.int64)
Stv["month"]=Stv["month"].astype(np.int64)
Stv.head()

,id,lat,long,Region,edgeUID_,osmId,dist,edge_uid,osm_refere,month,stv_total,stv_com_total,stv_rec_total,stv_com_ratio,stv_rec_ratio,year
0,100000671,37.79,-122.46,Bay Area,308723101,215845474,d4,308723101,215845474,8,420,20,400,0.05,0.95,2018
1,100000671,37.79,-122.46,Bay Area,308723101,215845474,d4,308723101,215845474,12,385,10,375,0.03,0.97,2018
2,100000671,37.79,-122.46,Bay Area,308723101,215845474,d4,308723101,215845474,1,395,25,370,0.06,0.94,2018
3,100000671,37.79,-122.46,Bay Area,308723101,215845474,d4,308723101,215845474,11,270,15,255,0.06,0.94,2018
4,100000671,37.79,-122.46,Bay Area,308723101,215845474,d4,308723101,215845474,4,430,30,400,0.07,0.93,2018


In [171]:
eco_strava=pd.merge(madt_fac, Stv, left_on=['id','lat','long', 'year','month'], right_on=['id','lat','long', 'year','month'], how='inner')

In [174]:
eco_strava.to_csv('C:/UC Berkeley Project Backup/AT Census/Processed Data/Eco_Strava_data.csv')

In [ ]:
# addd StreetLight Data Here